In [2]:
#https://giters.com/awslabs/dgl-ke/issues/232
import pandas as pd
import numpy as np

from collections import defaultdict
from datetime import datetime
dct = defaultdict(list)
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
%cd /content/gdrive/My\ Drive/
!pip3 install numpy==1.18.1
!pip3 install torch
!pip3 install dgl==0.4.3post2
!pip3 install dglke==0.1.2
!pip install mxnet
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64 && sudo ldconfig

Mounted at /content/gdrive/
/content/gdrive/My Drive
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [3]:
entity_emb = np.load('drug_repur/TransE_l2_NG_14/NG_TransE_l2_entity.npy')
rel_emb = np.load('drug_repur/TransE_l2_NG_14/NG_TransE_l2_relation.npy')
entities = pd.read_csv('drug_repur/entities.tsv', sep='\t', names=['id','entity'])
relations = pd.read_csv('drug_repur/relations.tsv', sep='\t', names=['id','rel'])


relation_list = relations.id.values.tolist()

entity2id = {}
id2entity = {}
for i in range(entities.shape[0]):
    entity2id[entities['entity'].values[i]] = i
    id2entity[i] = entities['entity'].values[i]
rel2id = {}
id2rel = {}
for i in range(relations.shape[0]):
    rel2id[relations['rel'].values[i]] = i
    id2rel[i] = relations['rel'].values[i]
    
with open('rel.list', 'w') as f:
    for item in relation_list:
        print(item)
        f.write("%s\n" % item)

#with open('head.list', 'w') as f:
 #   with open('./cui.list') as fh:
  #      for line in fh:
   #         cui = line.rstrip('\n')
    #        if cui in entity2id:
     #           f.write('%s\n' % entity2id[cui])

# head =  entities[(entities.entity.str.contains('Zinc')) | (entities.entity.str.contains('Ginseng'))]
# head_list = head.id.values.tolist()
# with open('head.list', 'w') as f:
#     for item in head_list:
#         f.write("%s\n" % item)

for idx, i in enumerate(relation_list):
    with open('./rel.list', 'w') as f:
        f.write("%s\n" % i)
 
tail = entities[(entities.entity.str.contains('Alzheimer'))]
tail_list = tail.id.values.tolist()
tail_list.pop(4)

with open('tail.list', 'w') as f:
    for item in tail_list:
        f.write("%s\n" % item)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [4]:
for idx, i in enumerate([1,3,11,12]):
    print(i)
    ## [1,3,11,12]
    with open('./rel.list', 'w') as f:
        f.write("%s\n" % i)
    OUTFILE='drug_repur/chemical_new_TransE_head' + str(id2rel[i]) + '.txt'
    #!dglke_predict --model_path ./drug_repur/TransE_l2_NG_6/ --format '*_r_t' --data_files rel.list tail.list --topK 10 --gpu 0 --exec_mode batch_head --output {OUTFILE}
    #!MXNET_ENGINE_TYPE=NaiveEngine DGLBACKEND=mxnet dglke_predict --model_path ./drug_repur/TransE_l2_NG_14/ --format '*_r_t' --data_files rel.list tail.list --topK 10 --gpu 0 --exec_mode 'batch_head' --output {OUTFILE}
    !MXNET_ENGINE_TYPE=NaiveEngine DGLBACKEND=mxnet dglke_predict --model_path ./drug_repur/TransE_l2_NG_14/ --format 'h_r_t' --data_files head_chemical_new.list rel.list tail.list --topK 10 --gpu 0 --exec_mode 'batch_head' --output {OUTFILE}

1
Using backend: mxnet
./drug_repur/TransE_l2_NG_14/config.json
{'dataset': 'NG', 'model': 'TransE_l2', 'emb_size': 400, 'max_train_step': 10000, 'batch_size': 2070, 'neg_sample_size': 30, 'lr': 0.01, 'gamma': 10.0, 'double_ent': False, 'double_rel': False, 'neg_adversarial_sampling': False, 'adversarial_temperature': 1.0, 'regularization_coef': 2e-06, 'regularization_norm': 3, 'emap_file': 'entities.tsv', 'rmap_file': 'relations.tsv'}
[05:15:05] ../src/engine/engine.cc:54: MXNet start using engine: NaiveEngine
Inference Done
The result is saved in drug_repur/chemical_new_TransE_headAFFECTS.txt
3
Using backend: mxnet
./drug_repur/TransE_l2_NG_14/config.json
{'dataset': 'NG', 'model': 'TransE_l2', 'emb_size': 400, 'max_train_step': 10000, 'batch_size': 2070, 'neg_sample_size': 30, 'lr': 0.01, 'gamma': 10.0, 'double_ent': False, 'double_rel': False, 'neg_adversarial_sampling': False, 'adversarial_temperature': 1.0, 'regularization_coef': 2e-06, 'regularization_norm': 3, 'emap_file': 'ent

In [5]:
id2entity = {}
for i in range(entities.shape[0]):
    id2entity[i] = entities['entity'].values[i]

id2rel = {}
for i in range(relations.shape[0]):
    id2rel[i] = relations['rel'].values[i]
    
#cui2name = dict(zip(instances['cui'], instances['name']))

#df = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})

## Filter Results

In [6]:
import csv
results = pd.read_csv('drug_repur/chemical_new_TransE_headAFFECTS.txt', sep='\t')
results.sort_values('score', ascending=False, inplace=True)
results = results[:1000]
df = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})
df_train = open('drug_repur/train.tsv')
df_train = csv.reader(df_train, delimiter="\t")
seen = set()
for key in df_train:
  seen.add((key[0],key[1],key[2]))

for index,key in df.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen:
    df.drop([index],inplace=True)



In [7]:
df[:30]

,head,rel,tail,score
45030,Rice,AFFECTS,Familial Alzheimer Disease (FAD),-9.152461
32290,Lactams,AFFECTS,Familial Alzheimer Disease (FAD),-9.200564
29470,Honey,AFFECTS,Familial Alzheimer Disease (FAD),-9.214219
45031,Rice,AFFECTS,Alzheimer Disease 7,-9.224507
45032,Rice,AFFECTS,Alzheimer's Disease,-9.236558
32291,Lactams,AFFECTS,Alzheimer Disease 7,-9.268093
39860,Pentylenetetrazole,AFFECTS,Familial Alzheimer Disease (FAD),-9.280295
29471,Honey,AFFECTS,Alzheimer's Disease,-9.282342
24400,Electrolytes,AFFECTS,Familial Alzheimer Disease (FAD),-9.285413
29472,Honey,AFFECTS,Alzheimer Disease 7,-9.288003


In [ ]:
df.to_csv('latest_chemicalnew_1.csv', index=False)

In [8]:
# import csv
# import pandas as pd
# from google.colab import drive
# drive.mount('/content/gdrive/',force_remount=True)
# import csv
# %cd /content/gdrive/My\ Drive/
# entities = pd.read_csv('drug_repur/entities.tsv', sep='\t', names=['id','entity'])
# relations = pd.read_csv('drug_repur/relations.tsv', sep='\t', names=['id','rel'])
# id2entity = {}
# for i in range(entities.shape[0]):
#     id2entity[i] = entities['entity'].values[i]

# id2rel = {}
# for i in range(relations.shape[0]):
#     id2rel[i] = relations['rel'].values[i]

# df_train = open('drug_repur/train.tsv')
# df_train = csv.reader(df_train, delimiter="\t")
# seen = set()
# for key in df_train:
#   seen.add((key[0],key[1],key[2]))

results = pd.read_csv('drug_repur/chemical_new_TransE_headTREATS.txt', sep='\t')
results.sort_values('score', ascending=False, inplace=True)
results = results.head(100)
df_treat = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})

for index,key in df_treat.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen:
    print(temp)
    df_treat.drop([index],inplace=True)


('gabapentin', 'TREATS', "Alzheimer's Disease")
('Ivermectin', 'TREATS', "Alzheimer's Disease")
('Imipramine', 'TREATS', "Alzheimer's Disease")
('Hydroxychloroquine', 'TREATS', "Alzheimer's Disease")
('Naltrexone', 'TREATS', "Alzheimer's Disease")
('Acitretin', 'TREATS', "Alzheimer's Disease")
('Antihistamines', 'TREATS', "Alzheimer's Disease")
('Minocycline', 'TREATS', "Alzheimer's Disease")
('Clonazepam', 'TREATS', "Alzheimer's Disease")
('topiramate', 'TREATS', "Alzheimer's Disease")
('Levetiracetam', 'TREATS', "Alzheimer's Disease")
('Amitriptyline', 'TREATS', "Alzheimer's Disease")
('quetiapine', 'TREATS', "Alzheimer's Disease")
('carvedilol', 'TREATS', "Alzheimer's Disease")
('aripiprazole', 'TREATS', "Alzheimer's Disease")


In [9]:
df_treat[:15]
df_treat.to_csv('chemical_new_latest2.csv', index=False)

In [10]:
# import csv
# # import pandas as pd
# # from google.colab import drive
# # drive.mount('/content/gdrive/',force_remount=True)

# %cd /content/gdrive/My\ Drive/
# entities = pd.read_csv('drug_repur/entities.tsv', sep='\t', names=['id','entity'])
# relations = pd.read_csv('drug_repur/relations.tsv', sep='\t', names=['id','rel'])
# id2entity = {}
# for i in range(entities.shape[0]):
#     id2entity[i] = entities['entity'].values[i]

# id2rel = {}
# for i in range(relations.shape[0]):
#     id2rel[i] = relations['rel'].values[i]

# df_train = open('drug_repur/train.tsv')
# df_train = csv.reader(df_train, delimiter="\t")
# seen = set()
# for key in df_train:
#   seen.add((key[0],key[1],key[2]))

results = pd.read_csv('drug_repur/chemical_new_TransE_headPREVENTS.txt', sep='\t')
results.sort_values('score', ascending=False, inplace=True)
results = results.head(100)
df_prevent = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})

for index,key in df_prevent.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen:
    print(temp)
    df_prevent.drop([index],inplace=True)


('Phenolics', 'PREVENTS', "Alzheimer's Disease")
('Rifampin', 'PREVENTS', "Alzheimer's Disease")
('small molecule', 'PREVENTS', "Alzheimer's Disease")
('Raloxifene', 'PREVENTS', "Alzheimer's Disease")
('pioglitazone', 'PREVENTS', "Alzheimer's Disease")
('Thioctic Acid', 'PREVENTS', "Alzheimer's Disease")
('Bile Acids', 'PREVENTS', "Alzheimer's Disease")
('Cannabinoids', 'PREVENTS', "Alzheimer's Disease")
('obovatol', 'PREVENTS', "Alzheimer's Disease")
('Fatty Acids, Monounsaturated', 'PREVENTS', "Alzheimer's Disease")
('Genistein', 'PREVENTS', "Alzheimer's Disease")
('Polysaccharides', 'PREVENTS', "Alzheimer's Disease")
('CP 113818', 'PREVENTS', "Alzheimer's Disease")
('rhyncophylline', 'PREVENTS', "Alzheimer's Disease")
('Fluoxetine', 'PREVENTS', "Alzheimer's Disease")
('Aspirin', 'PREVENTS', "Alzheimer's Disease")
('resveratrol', 'PREVENTS', "Alzheimer's Disease")
('polyphenols', 'PREVENTS', "Alzheimer's Disease")
('maysin', 'PREVENTS', "Alzheimer's Disease")
('cyclen', 'PREVENTS', "

In [11]:
df_prevent[:15]
df_prevent.to_csv('chemical_new_latest3.csv', index=False)

In [12]:
# import csv
# # import pandas as pd
# # from google.colab import drive
# # drive.mount('/content/gdrive/',force_remount=True)

# %cd /content/gdrive/My\ Drive/
# entities = pd.read_csv('drug_repur/entities.tsv', sep='\t', names=['id','entity'])
# relations = pd.read_csv('drug_repur/relations.tsv', sep='\t', names=['id','rel'])
# id2entity = {}
# for i in range(entities.shape[0]):
#     id2entity[i] = entities['entity'].values[i]

# id2rel = {}
# for i in range(relations.shape[0]):
#     id2rel[i] = relations['rel'].values[i]

# df_train = open('drug_repur/train.tsv')
# df_train = csv.reader(df_train, delimiter="\t")
# seen = set()
# for key in df_train:
#   seen.add((key[0],key[1],key[2]))

results = pd.read_csv('drug_repur/chemical_new_TransE_headCAUSES.txt', sep='\t')
results.sort_values('score', ascending=False, inplace=True)
results = results.head(100)
df_cause = results.replace({'head': id2entity, 'rel': id2rel, 'tail': id2entity})

for index,key in df_cause.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen:
    df_cause.drop([index],inplace=True)


In [13]:
df_cause.to_csv('chemical_new_latest4.csv', index=False)

In [ ]:

for index,key in df_prevent.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'Zinc' in key[0]:
    print(temp)

for index,key in df_cause.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'Zinc' in key[0]:
    print(temp)


for index,key in df_treat.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'Zinc' in key[0]:
    print(temp)

for index,key in df.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'Zinc' in key[0]:
    print(temp)

('Zinc', 'PREVENTS', "Alzheimer's Disease")
('Zinc', 'AFFECTS', "Alzheimer's Disease")


In [ ]:
for index,key in df_prevent.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'ginseng' in key[0]:
    print(temp)

for index,key in df_cause.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'ginseng' in key[0]:
    print(temp)


for index,key in df_treat.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'ginseng' in key[0]:
    print(temp)

for index,key in df.iterrows():
  temp = (key[0],key[1],key[2])
  if temp in seen and 'ginseng' in key[0]:
    print(temp)

In [ ]:
## ginseng related entity